In [105]:
import math
import numpy as np
from codecs import encode

In [106]:
alphabet = "abcdefghjiklmnopqrstuvxwyz"
alphabet_characters = [char for char in alphabet]

In [107]:
def forward_propagation(inputs, hidden_weights, hidden_bias, output_weights, output_bias):
  hidden_layer_activation = np.dot(inputs, hidden_weights) + hidden_bias
  hidden_layer_output = sigmoid_activation(hidden_layer_activation)
  output_layer_activation = np.dot(hidden_layer_output, output_weights) + output_bias
  predicted = sigmoid_activation(output_layer_activation)
  return predicted, hidden_layer_output

def backward_propagation(error, predicted, hidden_layer_output, output_weights):
  derivative_predicted = error * sigmoid_derivative(predicted)
  hidden_layer_error = derivative_predicted.dot(output_weights.T)
  derivative_hidden_layer = hidden_layer_error * sigmoid_derivative(hidden_layer_output)
  return derivative_predicted, derivative_hidden_layer

def update_weights(inputs, hidden_weights, hidden_bias, output_weights, output_bias, hidden_layer_output, derivative_predicted, derivative_hidden_layer, eta):
  output_weights += hidden_layer_output.T.dot(derivative_predicted) * eta
  output_bias += np.sum(derivative_predicted, axis=0, keepdims=True) * eta
  hidden_weights += inputs.T.dot(derivative_hidden_layer) * eta
  hidden_bias += np.sum(derivative_hidden_layer, axis=0, keepdims=True) * eta
  return output_weights, output_bias, hidden_weights, hidden_bias

def sigmoid_activation(x):
  return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
  return x * (1 - x)

def mlp_start_weights(mlp_input, mlp_hidden, mlp_output):
  hidden_weights = np.random.uniform(size=(mlp_input, mlp_hidden))
  hidden_bias = np.random.uniform(size=(1, mlp_hidden))
  output_weights = np.random.uniform(size=(mlp_hidden, mlp_output))
  output_bias = np.random.uniform(size=(1, mlp_output))
  return hidden_weights, hidden_bias, output_weights, output_bias

In [126]:
identity_n = 26
encoder_input = np.identity(identity_n)
encoder_output = np.roll(np.identity(identity_n), identity_n//2, axis=0)

eta = 0.01
stop_value = 0.0001
max_iter = 500000
mlp_input_hidden_output = (identity_n, int(math.log(identity_n,2)), identity_n) # Duas camadas de entrada, duas intermediárias, uma de saída

error_output_layer = math.inf
input_size, hidden_size, output_size = mlp_input_hidden_output
hidden_weights, hidden_bias, output_weights, output_bias = mlp_start_weights(input_size, hidden_size, output_size)

# Treinamento do MLP
for _ in range(max_iter):
  if error_output_layer < stop_value: # Cancela treinamento se erro ja chegou no valor de parada
    break
  
  predicted, hidden_layer_output = forward_propagation(encoder_input, hidden_weights, hidden_bias, output_weights, output_bias)
  error = encoder_output - predicted
  derivative_predicted, derivative_hidden_layer = backward_propagation(error, predicted, hidden_layer_output, output_weights)
  output_weights, output_bias, hidden_weights, hidden_bias = update_weights(encoder_input, hidden_weights, hidden_bias, output_weights, output_bias, hidden_layer_output, derivative_predicted, derivative_hidden_layer, eta)
  error_output_layer = abs(error[-1][0])


In [127]:
string_to_encode = "hygenfrpergzrffntr"
string_to_encode_characters = [char for char in string_to_encode]
letter_positions = np.searchsorted(alphabet_characters, string_to_encode_characters)

word = np.zeros((len(letter_positions), len(alphabet_characters)))
for index, letter in enumerate(letter_positions):
  word[index][letter] = 1

In [128]:
predicted, hidden_layer_output = forward_propagation(word, hidden_weights, hidden_bias, output_weights, output_bias)

In [129]:
print("Original message:  %s" % string_to_encode)
print("Predicted message: ", end='')
for one_hot_encoded_letter in np.round(predicted):
  for encoded_letter, letter in zip(one_hot_encoded_letter, alphabet_characters):
    if encoded_letter and letter:
      print(letter, end='')

print("\nRight message:\t   %s" % encode(string_to_encode, "rot_13"))


Original message:  hygenfrpergzrffntr
Predicted message: ultrasecretmessage
Right message:	   ultrasecretmessage
